In [1]:
import pandas as pd
import numpy as np
import glob
from tqdm import tqdm
from bs4 import BeautifulSoup
from requests import get
import multiprocessing
import lxml
import csv

from scraperFuncs import *

In [2]:
winesRaw = pd.read_pickle('data/scraped/scrape.pickle')
winesRaw = winesRaw.reset_index(drop=True)
print(f"df_wine_list: {winesRaw.shape}")

# Read in the downloaded filenames
#wine_pages = glob.glob('data/wine_pages/wine_*', recursive=True)
#wine_pages = np.sort(wine_pages)

df_wine_list: (130348, 5)


In [3]:
winesRaw.head()

,name,image_path,price,url,raw_html
0,Dom Perignon Lenny Kravitz Limited Edition wit...,/product/images/fl_progressive/lmgmud1xsenlouw...,199.97,/product/Dom-Perignon-Lenny-Kravitz-Limited-Ed...,"<div class=""prodItem_wrap"">\n<div class=""prodI..."
1,Louis Roederer Cristal Brut with Two Flutes an...,/product/images/fl_progressive/iqec7e0nf6mzyof...,329.99,/product/Louis-Roederer-Cristal-Brut-with-Two-...,"<div class=""prodItem_wrap"">\n<div class=""prodI..."
2,Laurent-Perrier Cuvee Rose,/product/images/fl_progressive/10521.jpg,79.99,/product/Laurent-Perrier-Cuvee-Rose/10521,"<div class=""prodItem_wrap"">\n<div class=""prodI..."
3,Piper-Heidsieck Cuvee Brut in Travel Case with...,/product/images/fl_progressive/aqi87aqobd3zc56...,79.99,/product/Piper-Heidsieck-Cuvee-Brut-in-Travel-...,"<div class=""prodItem_wrap"">\n<div class=""prodI..."
4,Clarendon Hills Astralis Syrah 2011,/product/images/fl_progressive/qajrqr4d6ttn2pf...,149.99,/product/Clarendon-Hills-Astralis-Syrah-2011/5...,"<div class=""prodItem_wrap"">\n<div class=""prodI..."


In [8]:
wines = winesRaw.copy()

wines.reset_index(inplace=True)

### Scrape and Extract Descriptions

In [ ]:
%%time

all_urls = winesRaw['url']
all_urls = all_urls[:40]

p = Pool(10)
url_list, desc_list = zip(*p.map(scrapeAndExtract, all_urls))
p.terminate()
p.join()

In [ ]:
%%time

all_urls = winesRaw['url']
#all_urls = all_urls[:10]
#all_urls = ["/product/Crossbarn-by-Paul-Hobbs-Napa-Valley-Cabernet-Sauvignon-375ml-half-bottle-2016/518519"]


def mp_handler():
    #p = multiprocessing.Pool(20)
    with multiprocessing.Pool(10) as p:
        with open('data/scraped/descriptions.csv', 'a') as f:
            writer= csv.writer(f, lineterminator = '\n', 
                               delimiter="|", 
                               quoting=csv.QUOTE_NONNUMERIC)
            quoting=csv.QUOTE_NONNUMERIC
            for result in p.imap(scrapeAndExtract, all_urls):
                print(result[0])
                #print(result[1][:5])
                try: 
                    desc = result[1].strip('\"')
                except Exception as e:
                    desc = str(e)
                writer.writerow([result[0],desc])
                # (filename, count) tuples from worker
                #f.write(f"URL:{result[0]}|DESC:{result[1]}")
            
            
mp_handler()

p.terminate()
p.join()

In [ ]:
%%time

all_urls = winesRaw['url']
#all_urls = all_urls[:40]

p = Pool(10)
url_list, rawHTML_list = zip(*p.map(scrapeHTML, all_urls))
p.terminate()
p.join()

In [ ]:
%%time

p = Pool(16)
url_list, rawHTML_list = zip(*p.map(extractDescription, rawHTML_list))
p.terminate()
p.join()

In [ ]:
descLengths = [len(desc[1]) for desc in descriptions]
descLengths = pd.Series(descLengths)
descLengths = descLengths[descLengths < 100]
print(len(descLengths))

In [ ]:
url = "/product/dom-perignon-lenny-kravitz-limited-edition-with-gift-box-2008/544740"
html = get("http://www.wine.com"+url).text

soup = BeautifulSoup(html, "lxml")
#allText = soup.prettify()
#soup2 = BeautifulSoup(allText, "lxml")



In [ ]:
soup.find("div", {"itemprop":"description"})\
    .find("div", {"class":"viewMoreModule_text"}).text

In [ ]:
html = winesRaw['raw_html'][0]
soup = BeautifulSoup(html, "lxml")
soup.find("div", {"itemprop":"description"})\
    .find("div", {"class":"viewMoreModule_text"}).text

In [ ]:
soup.find("div",{"itemprop":"description"})

In [ ]:
dir(soup)

In [ ]:
'<div class="viewMoreModule_text">

In [ ]:
def scraper(url):
    print(url)
    try:
        html = get("http://www.wine.com"+url).text
        soup = BeautifulSoup(html, "html.parser")
        allText = soup.prettify()
    except Exception as e:
        return (url, e)
    
    try:
        desc = allText.split("pipWineNotes_copy")[1]\
                      .split("viewMoreModule_text")[1]\
                      .split(">\n         <p>\n          ")[1]\
                      .split("\n         </p")[0]
    except IndexError:
        allText.split("pipWineNotes_copy")[1]\
               .split("viewMoreModule_text")[1]\
               .split("</div>")[0]\
        
        return allText
    except Exception as e:
        desc = str(e)
        
    return (url, desc)

page = "Veuve-Clicquot-Yellow-Label-Brut/528"
allText = scraper(page)

In [ ]:
allText.split("pipWineNotes_copy")[1]\
                      .split("viewMoreModule_text")[1]\
                      .split("</div>")[0]

In [ ]:
allText

In [ ]:
import requests
from bs4 import BeautifulSoup
from queue import Queue, Empty
from concurrent.futures import ThreadPoolExecutor
from urllib.parse import urljoin, urlparse


class MultiThreadScraper:

    def __init__(self, base_url):

        self.base_url = base_url
        self.root_url = '{}://{}'.format(urlparse(self.base_url).scheme, urlparse(self.base_url).netloc)
        self.pool = ThreadPoolExecutor(max_workers=20)
        self.scraped_pages = set([])
        self.to_crawl = Queue()
        self.to_crawl.put(self.base_url)

    def parse_links(self, html):
        soup = BeautifulSoup(html, 'html.parser')
        links = soup.find_all('a', href=True)
        for link in links:
            url = link['href']
            if url.startswith('/') or url.startswith(self.root_url):
                url = urljoin(self.root_url, url)
                if url not in self.scraped_pages:
                    self.to_crawl.put(url)

    def scrape_info(self, html):
        return

    def post_scrape_callback(self, res):
        result = res.result()
        if result and result.status_code == 200:
            self.parse_links(result.text)
            self.scrape_info(result.text)

    def scrape_page(self, url):
        try:
            res = requests.get(url, timeout=(3, 30))
            return res
        except requests.RequestException:
            return

    def run_scraper(self):
        while True:
            try:
                target_url = self.to_crawl.get(timeout=60)
                if target_url not in self.scraped_pages:
                    print("Scraping URL: {}".format(target_url))
                    self.scraped_pages.add(target_url)
                    job = self.pool.submit(self.scrape_page, target_url)
                    job.add_done_callback(self.post_scrape_callback)
            except Empty:
                return
            except Exception as e:
                print(e)
                continue
if __name__ == '__main__':
    s = MultiThreadScraper("http://www.example.co.uk")
    s.run_scraper()

In [ ]:
winesClean = winesRaw.copy()

# Clean in duplicate indices
winesClean.reset_index(inplace=True, drop=True)

# Scrape descriptions from site
ix = []
name = []
descList = []
for i in enumerate(tqdm(winesRaw['url'])):
    #print(i[0])
    try:
        html = get("http://www.wine.com"+i[1]).text
        soup = BeautifulSoup(html)
        allText = soup.prettify()
        desc = allText.split("pipWineNotes_copy")[1]\
                      .split("viewMoreModule_text")[1]\
                      .split(">\n         <p>\n          ")[1]\
                      .split("\n         </p")[0]
    except IndexError:
        #print("oops")
        desc = "IndexError"
        
    
    ix.append()
    name.append(winesRaw['name']i[0])
    descList.append(desc)
    
    if i[0] % 1000 == 0:
        pd.Series(descList).to_csv(f"data/scraped/descriptions/descriptionsUpTo{i[0]}")

In [ ]:
soup = BeautifulSoup(i[1])
allText = soup.prettify()

allText.split("pipWineNotes_copy")[1]\
                     # .split("viewMoreModule_text")[1]\
                     # .split(">\n         <p>\n          ")[1]\
                     # .split("\n         </p")[0]

In [ ]:
winesClean.head()

In [ ]:
for i in enumerate(tqdm(list([1,2,3,4]))):
    print(i)

In [ ]:
for i in enumerate(tqm(winesRaw['raw_html'])):
    #print(html)
    try:
        desc = html.split('<div class="viewMoreModule_text"><p>')[1].split('<')[0]
    except IndexError:
        desc = html.split('<div class="viewMoreModule_text">')[1].split('<')[0]
    
    print(desc)
    break

In [ ]:
# Read, decode, and save wine descriptions & names
names = []
descriptions = []
for page in tqdm(wine_pages):
    #print(page)
    with open(page, "rb") as f:
        #wine = BeautifulSoup(f.read(), parse_only=)
        wine = f.read().decode('utf-8')
    try:  
        names.append(wine.split('><title>')[1].split('|')[0])
    except IndexError:
        continue
    try:
        descriptions.append(wine.split('<div class="viewMoreModule_text"><p>')[1].split('<')[0])
    except IndexError:
        descriptions.append(wine.split('<div class="viewMoreModule_text">')[1].split('<')[0])
        
pd.to_pickle((names,descriptions), path='data/scraped/cleaned_data.pickle')

In [ ]:
# Extract Name, Description, and Price information from data
# Some of this is redundant, but it works 

OUTPUT_PATH = 'data/scraped/names_prices_descriptions.pickle'
descriptions = pd.read_pickle('data/scraped/descriptions.pickle')

counter = 0
list1 = []
list2 = []
list3 = []
for name in tqdm(descriptions[0]):
    price = df_scraped[df_scraped['name'] == name.strip()]['price'].values
    desc = descriptions[1][counter]
    list1.append(name.strip())
    list2.append(price)
    list3.append(desc)
    counter += 1
    
pd.DataFrame({'name'        : list1,
              'price'       : list2,
              'description' : list3})\
    .to_pickle(OUTPUT_PATH)